# SBB Hack Class - Backend for Retrieving Price Offers of SBB Trips

HackZurich 2019

This notebook was used for testing and initial assembling of the backend. The final version is stored in a separate py-file.
Main learnings: working with GET and POST requests to retrieve different data from the SBB App. The ouptut was then sent to the Frontend by FLASK.

Suggested improvements:
- better JSON parsing (see z_test.ipynb)
- error handling (try-except) for debugging
- too many variables stored in class object (input REB)

API
- [Docu](https://b2p.app.sbb.ch/docs/index.html)
- [Swagger](https://b2p.app.sbb.ch/swagger-ui.html#/)
- [Example App](https://booking.app.trasier.com/app/)


Super helpful site: [Translate CURL commands](https://curl.trillworks.com/#python)

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Create-Base-Class-With-all-The-Functions" data-toc-modified-id="Create-Base-Class-With-all-The-Functions-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Create Base Class With all The Functions</a></span><ul class="toc-item"><li><span><a href="#Test-no-offer-available" data-toc-modified-id="Test-no-offer-available-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Test no offer available</a></span></li></ul></li></ul></div>

In [1]:
from datetime import datetime, timedelta
import requests
import json
import dateparser

In [2]:
"""Get the access token and set headers"""

data = {'grant_type': 'client_credentials',
        'client_id': 'xxx',
        'client_secret': 'xxx'}

response = requests.post('https://sso-int.sbb.ch/auth/realms/SBB_Public/protocol/openid-connect/token', data=data)

# Parse Token
response_dict = json.loads(response.text)
access_token = response_dict['access_token']

# Set headers
headers = {
    'Accept': '*/*',
    'Accept-Language': 'de',
    'X-Contract-Id': 'HAC222P',
    'X-Conversation-Id': 'cafebabe-0815-4711-1234-ffffdeadbeef',
    'Authorization': 'Bearer ' + access_token,
}

## Create Base Class With all The Functions

In [3]:
travel_date = '2019-10-27'
travel_time = '10:00'
travel_dest_name = 'Montreux'
travel_orig_name = 'Zurich'

In [4]:
class SurpriseRequest:
    """ Class for a surprise trip request."""
    
    def __init__(self, 
                 trainType='IR;ICE/TGV/RJ,EC/IC',
                 arrivalDeparture='ED',
                 ipExtensionTicket='false',
                 passengers='paxa;42;half-fare'):
        """ Initialize Surprise Request instance."""

        self._trainType = trainType
        self._arrivalDeparture = arrivalDeparture
        self._ipExtensionTicket=ipExtensionTicket
        self._passengers=passengers
        
    
#     def __repr__(self):
#         """Function to output the characteristics of the instance."""
#         try:
#             date = str('date ' + self._travel_date)
#             time = str('time ' + self._travel_time)
#             orig = str('origin ' + self._travel_orig_name)

#             return str(date + " / " + time + " / " + orig)
    

    def go_for_offer(self, 
                     travel_date=travel_date, 
                     travel_time=travel_time,
                     travel_dest_name = travel_dest_name,
                     travel_orig_name = travel_orig_name):
        """Chain many functions to try if you can return an offer price."""
        
        self._travel_date = travel_date
        self._travel_time = travel_time
        self._travel_dest_name = travel_dest_name 
        self._travel_orig_name = travel_orig_name
        
        self.get_location_ids()
        self.get_start_trip_id()
        self.get_return_trip_id()
        self.get_offers()
        return self.return_offer_price()        
        
    
    
    def get_location_ids(self):
        """Get the location ids from the name inputs."""

        loc_id_list = []
        for location in [self._travel_dest_name, self._travel_orig_name]:
            loc_params = (
                ('name', location),
            )

            loc_data = requests.get('https://b2p-int.api.sbb.ch/api/locations', 
                                    headers=headers, params=loc_params)
            loc_data_list = json.loads(loc_data.text)
            loc_id = loc_data_list[0]['id']
            loc_id_list.append(loc_id)

        self._travel_dest = loc_id_list[0]
        self._travel_orig = loc_id_list[1]


    def get_start_trip_id(self):
        """Get the initial journey Id."""


        start_trip_params = (
            ('arrivalDeparture', self._arrivalDeparture),
            ('date', self._travel_date),
            ('destinationId', self._travel_dest),
            ('originId', self._travel_orig),
            ('time', self._travel_time),
            ('trainType', self._trainType),
        )

        start_trip = requests.get('https://b2p-int.api.sbb.ch/api/trips', 
                                  headers=headers, params=start_trip_params)
        start_trip_as_list = json.loads(start_trip.text)

        self._start_trip_id = start_trip_as_list[0]['tripId']


    def get_return_trip_id(self, delta_hours=5):
        """Get the initial journey Id."""
        self._deta_hours = delta_hours

        datetime_travel = dateparser.parse(travel_date + ',' + travel_time)
        datetime_return = datetime_travel + timedelta(hours=delta_hours)
        return_time = str(datetime_return.time())[:-3]

        return_trip_params = (
            ('arrivalDeparture', self._arrivalDeparture),
            ('date', self._travel_date),
            ('destinationId', self._travel_orig),
            ('originId', self._travel_dest),
            ('time', return_time),
            ('trainType', self._trainType),
        )

        return_trip = requests.get('https://b2p-int.api.sbb.ch/api/trips', 
                                   headers=headers, params=return_trip_params)
        return_trip_as_list = json.loads(return_trip.text)
        
        self._return_trip_id = return_trip_as_list[0]['tripId']
                 
                 
    def get_offers(self):
        """Get price offer for full itinerary."""
        
        offer_params = (
            ('ipExtensionTicket', self._ipExtensionTicket),
            ('passengers', self._passengers),
            ('returnTripId', self._return_trip_id),  
            ('tripId', self._start_trip_id),
        )
                 
        offer = requests.get('https://b2p-int.api.sbb.ch/api/trip-offers', 
                              headers=headers, params=offer_params)
        self._offer_as_list = json.loads(offer.text) 
        
        product_id_list = []
        for offer in self._offer_as_list:
            offer_id = offer['offers'][0]['productId']
            product_id_list.append(offer_id)
            
        product_id_set = set(product_id_list)
        
        if 4004 in product_id_set:
            for offer in self._offer_as_list:
                if offer['offers'][0]['productId'] == 4004:
                    self._offer_price = offer['offers'][0]['price'] *2
                    break            

            for offer in self._offer_as_list:
                if offer['offers'][0]['productId'] == 125:
                    self._full_price = offer['offers'][0]['price'] *2
                    break
        else:
            self._offer_price = "no supersaver"
        
        print(self._offer_price)
        
    def return_offer_price(self):
        """Return an offer price if both trips have Supersaver Price."""
        
        if isinstance(self._offer_price, str):
            print(True)
            
            return {'offer_price': None,
                    'rebate': None,
                    'rebate_pct': None}
        
        else:
            self._rebate = self._full_price - self._offer_price
            self._rebate_ptc = round(self._rebate / self._full_price, 2)
            
            return {'offer_price': self._offer_price,
                    'rebate': self._rebate,
                    'rebate_pct': self._rebate_ptc}

In [5]:
request1=SurpriseRequest()

# Check
# request1

In [6]:
results = request1.go_for_offer()

4000


In [7]:
results

{'offer_price': 4000, 'rebate': 10000, 'rebate_pct': 0.71}

### Test no offer available

Set actual time als startdate to be sure there is no supersaver ticket available.

In [8]:
now = datetime.now()
travel_date = str(now.date())
travel_time = str(now.time())[:5]

In [9]:
request1.go_for_offer(travel_date=travel_date, travel_time=travel_time)

no supersaver
True


{'offer_price': None, 'rebate': None, 'rebate_pct': None}

---